In [64]:
#@title Bibliotecas padrões
import pandas as pd
import pandas.io.sql as sqlio
import sys
import psycopg2
from ydata_profiling import ProfileReport
import numpy as np
from botocore import UNSIGNED
from botocore.config import Config
from urllib.parse import quote_plus
from pyathena import connect
import boto3
import boto3.session
from botocore import exceptions
import seaborn as sns
import plotly.express as px

#Carregando as bibliotecas necessárias para a análise estatística
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8, 8)
import statsmodels.api as sm
import statsmodels.tsa as tsa
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pmdarima as pm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_process import arma_generate_sample

In [65]:
#ponte de conexão com o redshift
connection = psycopg2.connect(dbname="dw_amorsaude",
                  user="gabriel_guilherme",
                  password="PgQ1yBg9hzHJfmc",
                  host ="amorsaude-cluster-redshift.cmc4ev2oyczd.us-east-1.redshift.amazonaws.com",
                  port = 5439)

In [66]:
#query
df_emails = '''with ticket as (
select tccarm.id_paciente as paciente_id, sum(tccarm.valor_pago)
	/count(*) as ticket_exames_procedimentos  
from tb_consolidacao_contas_a_receber_modelagem tccarm
left join stg_pacientes sp on sp.id = tccarm .id_paciente
where tccarm.data_pagamento between  current_date - 360 and current_date -1
and tccarm.nomegrupo not in ('Consultas', 'Retorno')
group by tccarm.id_paciente),
--separando os pacientes com atendimento maior que 3 e com atendimento nos últimos 3 meses.
pacientes as ( 
select sa.paciente_id as paciente_id1, sp.cpf as cpf , sp.email as email, sp.nascimento as data_nascimento,
DATEDIFF(year, sp.nascimento, CURRENT_DATE) AS idade,
count(sa.id) as qtde_atendimento, (max(sa."data") - current_date)*-1 as teste
from stg_atendimentos sa
left join stg_pacientes sp on sp.id = sa.paciente_id
where sa."data" between current_date - 360 and current_date -1
group by sa.paciente_id, sp.cpf , sp.email, sp.nascimento, DATEDIFF(year, sp.nascimento, CURRENT_DATE)
having count(sa.id) >=3
and (max(sa."data") - current_date)*-1 <=90
and DATEDIFF(year, sp.nascimento, CURRENT_DATE) >=18)
select paciente_id1, cpf, email, qtde_atendimento,  REPLACE(REPLACE(REPLACE(REPLACE(sum(ticket_exames_procedimentos)
	::text,'$','R$ '),',','|'),'.',','),'|','.') as ticket
from pacientes
inner join ticket on paciente_id = paciente_id1
where email is not null
	and cpf not in (
	select sf.cpf  
	from stg_usuarios su 
inner join stg_funcionarios sf on sf.id = su.id_relativo
where su.id in (119268,121109,121110,121113,121118,121121,121127,123518,
123886,162551,175414,178492,178590,184316,186510,149995324,149995326,149995330,
149995331,149995333,149998556,149999312,150000265,150000266,150000430,150000916,
150003574,150007761,150011688,150013014,150013015,150014695,150015275,150018241,
150018335,150018427,150018429,150018486,150020671,150024965,150025618,150026304,
160028819,160029416,160030289,160030950,160033339,160033362,160034917,160034918,
160035878,160038304,160038315,160038356,160039962,160040036,160042001,160042002,
160043459,160043690,160043943,160045111,160045792,160045795,160047148,160047566,
160047958,160049338,160049340,160049343,160049531,160052055,160052135,160054093,
160054100,160054107,160055826,160061181,160061189,160062661,160065149,160065987,
160067193,160070254,160070255,160072031,160075916,160077331,160078580,160079583,
160083281,160083403,160084850,160085294,160085356,160085783,160086297,160086575,
160086933,160090033,160090436,160090486,160090504,160090755,160090763,160092141,
160092143,160092546,160092547,160092548,160092549,160094957,160094965,160094969,
160095249,160101568,160102076,160103282,160104753,160106893,160109783,160109784,160109786))
--and cpf = '29542346875' --validação por paciente
group by paciente_id1, cpf, email, qtde_atendimento, ticket_exames_procedimentos
--having qtde_atendimento > 5
order by ticket_exames_procedimentos desc'''

In [67]:
df = pd.read_sql_query(df_emails,con=connection)
df

/tmp/ipykernel_20154/3073521440.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(df_emails,con=connection)


,paciente_id1,cpf,email,qtde_atendimento,ticket
0,60830958.0,70887818242,carmenvalentinabr@gmail.com,5,"3500,0000"
1,12092145.0,85320536887,,5,"3250,0000"
2,61711266.0,06087055193,vane.vs222@gmail.com,6,"2862,5000"
3,58293305.0,32232687821,palomaborges06@gmail.com,4,"2800,0000"
4,65821529.0,28770469822,,3,"2400,0000"
...,...,...,...,...,...
449671,58868473.0,35250461034,carmenbaraibar@hotmail.com,4,"0,0000"
449672,10508354.0,48303811835,,6,"0,0000"
449673,64088883.0,91411890434,luajlima77@gmail.com,10,"0,0000"
449674,65655262.0,25954204888,gilmaraaplibano@gmail.com,21,"0,0000"


In [68]:
#pip install validate_email

In [69]:
from validate_email  import validate_email

In [70]:
# validando os registros de email
valid_emails = []
for index, row in df.iterrows():
    email = row['email']
    if validate_email(email):
        valid_emails.append(index)

In [71]:
valid_df = df.loc[valid_emails]

In [72]:
valid_df

,paciente_id1,cpf,email,qtde_atendimento,ticket
0,60830958.0,70887818242,carmenvalentinabr@gmail.com,5,"3500,0000"
2,61711266.0,06087055193,vane.vs222@gmail.com,6,"2862,5000"
3,58293305.0,32232687821,palomaborges06@gmail.com,4,"2800,0000"
5,62995008.0,34734593809,oliveira.venndas@me.com,5,"2400,0000"
6,11886836.0,09597234807,edinasmrocha@yahoo.com.br,11,"2250,0000"
...,...,...,...,...,...
449670,61859773.0,27567715899,neiaa_monteiro@outlook.com,6,"0,0000"
449671,58868473.0,35250461034,carmenbaraibar@hotmail.com,4,"0,0000"
449673,64088883.0,91411890434,luajlima77@gmail.com,10,"0,0000"
449674,65655262.0,25954204888,gilmaraaplibano@gmail.com,21,"0,0000"


In [73]:
valid_df_150 = valid_df[:150000]
valid_df_150

,paciente_id1,cpf,email,qtde_atendimento,ticket
0,60830958.0,70887818242,carmenvalentinabr@gmail.com,5,"3500,0000"
2,61711266.0,06087055193,vane.vs222@gmail.com,6,"2862,5000"
3,58293305.0,32232687821,palomaborges06@gmail.com,4,"2800,0000"
5,62995008.0,34734593809,oliveira.venndas@me.com,5,"2400,0000"
6,11886836.0,09597234807,edinasmrocha@yahoo.com.br,11,"2250,0000"
...,...,...,...,...,...
181897,15257633.0,01985943727,schirlenemaster@hotmail.com,9,"36,9166"
181899,58593281.0,75319683804,marcelino.moreira201020@gmail.com,6,"36,9166"
181900,12318468.0,03883895911,florchiafre@gmail.com,10,"36,9166"
181901,64780969.0,11031912681,nataliafernandati@gmail.com,5,"36,9166"


In [74]:
from openpyxl import Workbook

In [75]:
nome_do_arquivo = 'dados.xlsx'
valid_df_150.to_excel(nome_do_arquivo, index=False)  # O argumento 'index=False' evita salvar o índice do DataFrame no Excel